In [130]:
import json, csv, os
import pandas as pd
import numpy as np
from scipy.stats import poisson

total value left by age?

In [131]:
transfer_values = pd.read_csv('player_valuations.csv')

# generate a list of values for 'date' in the csv
transfer_values['date'].unique()

transfer_values.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1
3,6893,2003-12-15,900000,984,GB1
4,10,2004-10-04,7000000,398,IT1


In [132]:
players = pd.read_csv('players.csv')

players.head()  

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [133]:
transfer_values['date'] = pd.to_datetime(transfer_values['date'])

this_year = transfer_values[transfer_values['date'].dt.year == 2024]

# if a player has multiple evaluations in the same year, take the most recent
this_year = this_year.sort_values('date').drop_duplicates('player_id', keep='last')

this_year.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
470293,745075,2024-01-10,25000,89,L1
470313,943560,2024-01-17,75000,873,GB1
470306,590410,2024-01-17,75000,371,SC1
470305,352815,2024-01-17,150000,2420,PO1
470309,703626,2024-01-17,100000,1237,GB1


In [134]:
def get_full_year_evals(year):
    this_year = transfer_values[transfer_values['date'].dt.year == year]
    this_year = this_year.sort_values('date').drop_duplicates('player_id', keep='last')
    this_year = this_year.merge(players, on='player_id')
    this_year['date_of_birth'] = pd.to_datetime(this_year['date_of_birth'])
    this_year['age'] = (this_year['date'] - this_year['date_of_birth']).dt.days // 365
    return this_year

In [135]:
# merge the two dataframes on the player_id column
this_year = pd.merge(this_year, players, how='left', on='player_id')

this_year.head()

,player_id,date,market_value_in_eur_x,current_club_id_x,player_club_domestic_competition_id,first_name,last_name,name,last_season,current_club_id_y,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur_y,highest_market_value_in_eur
0,745075,2024-01-10,25000,89,L1,David,Preu,David Preu,2024,89,...,right,172.0,2025-06-30 00:00:00,MGSC,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/david-preu/pro...,L1,1. FC Union Berlin,25000.0,50000.0
1,943560,2024-01-17,75000,873,GB1,Franco,Umeh-Chibueze,Franco Umeh-Chibueze,2023,873,...,left,NaN,2025-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/franco-umeh-ch...,GB1,Crystal Palace Football Club,75000.0,75000.0
2,590410,2024-01-17,75000,371,SC1,Josh,Clarke,Josh Clarke,2024,371,...,NaN,188.0,2027-06-30 00:00:00,CSG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/josh-clarke/pr...,SC1,The Celtic Football Club,75000.0,75000.0
3,352815,2024-01-17,150000,2420,PO1,Dénis,Duarte,Dénis Duarte,2023,2420,...,right,189.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/denis-duarte/p...,PO1,Vitória Sport Clube,150000.0,1000000.0
4,703626,2024-01-17,100000,1237,GB1,Cameron,Peupion,Cameron Peupion,2023,1237,...,right,175.0,2025-06-30 00:00:00,Cove Agency,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/cameron-peupio...,GB1,Brighton and Hove Albion Football Club,100000.0,150000.0


In [136]:
# convert date of birth into datetime
this_year['date_of_birth'] = pd.to_datetime(this_year['date_of_birth'])

# calculate age
this_year['age'] = (this_year['date'] - this_year['date_of_birth']).dt.days // 365

this_year.head()

,player_id,date,market_value_in_eur_x,current_club_id_x,player_club_domestic_competition_id,first_name,last_name,name,last_season,current_club_id_y,...,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur_y,highest_market_value_in_eur,age
0,745075,2024-01-10,25000,89,L1,David,Preu,David Preu,2024,89,...,172.0,2025-06-30 00:00:00,MGSC,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/david-preu/pro...,L1,1. FC Union Berlin,25000.0,50000.0,19.0
1,943560,2024-01-17,75000,873,GB1,Franco,Umeh-Chibueze,Franco Umeh-Chibueze,2023,873,...,NaN,2025-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/franco-umeh-ch...,GB1,Crystal Palace Football Club,75000.0,75000.0,18.0
2,590410,2024-01-17,75000,371,SC1,Josh,Clarke,Josh Clarke,2024,371,...,188.0,2027-06-30 00:00:00,CSG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/josh-clarke/pr...,SC1,The Celtic Football Club,75000.0,75000.0,19.0
3,352815,2024-01-17,150000,2420,PO1,Dénis,Duarte,Dénis Duarte,2023,2420,...,189.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/denis-duarte/p...,PO1,Vitória Sport Clube,150000.0,1000000.0,29.0
4,703626,2024-01-17,100000,1237,GB1,Cameron,Peupion,Cameron Peupion,2023,1237,...,175.0,2025-06-30 00:00:00,Cove Agency,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/cameron-peupio...,GB1,Brighton and Hove Albion Football Club,100000.0,150000.0,21.0


In [137]:
# now calcuate the total aggregate value of players of each age
age_val = this_year.groupby('age')['market_value_in_eur_x'].sum()

# now calcuate the total number of players of each age
# print(this_year.groupby('age')['market_value_in_eur_x'].count())

# now calcuate the average value of players of each age
# print(this_year.groupby('age')['market_value_in_eur_x'].mean())

age_val

age
16.0      31675000
17.0     346050000
18.0     546435000
19.0    1408995000
20.0    1900275000
21.0    2797710000
22.0    3501025000
23.0    3771775000
24.0    4888975000
25.0    4300425000
26.0    4434450000
27.0    3502650000
28.0    2552650000
29.0    1822100000
30.0    1668750000
31.0    1222500000
32.0     625125000
33.0     359875000
34.0     168550000
35.0      97975000
36.0      50800000
37.0      28150000
38.0      26875000
39.0       3550000
40.0       1825000
41.0       2150000
43.0        100000
Name: market_value_in_eur_x, dtype: int64

In [138]:
# okay, so we see a player's value increases with time, but younger players are still worth more on average
# in order to measure when a player's effectiveness peaks (to compare it with value), we need to calculate the total minutes of players of each age

appearances = pd.read_csv('appearances.csv')
appearances.shape


(1618667, 13)

In [139]:
# merge the two dataframes on the player_id column, keeping in mind one player will have multiple appearances
appearances = pd.merge(this_year, appearances, how='left', on='player_id')
appearances.head()

,player_id,date_x,market_value_in_eur_x,current_club_id_x,player_club_domestic_competition_id,first_name,last_name,name,last_season,current_club_id_y,...,player_club_id,player_current_club_id,date_y,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,745075,2024-01-10,25000,89,L1,David,Preu,David Preu,2024,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,943560,2024-01-17,75000,873,GB1,Franco,Umeh-Chibueze,Franco Umeh-Chibueze,2023,873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,590410,2024-01-17,75000,371,SC1,Josh,Clarke,Josh Clarke,2024,371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,352815,2024-01-17,150000,2420,PO1,Dénis,Duarte,Dénis Duarte,2023,2420,...,2420.0,2420.0,2018-02-24,Dénis Duarte,PO1,0.0,0.0,0.0,0.0,90.0
4,352815,2024-01-17,150000,2420,PO1,Dénis,Duarte,Dénis Duarte,2023,2420,...,2420.0,2420.0,2018-03-04,Dénis Duarte,PO1,0.0,0.0,0.0,0.0,90.0


In [140]:
# calculate the total minutes played by each age group
age_mins = appearances.groupby('age')['minutes_played'].sum()

type(age_mins)

pandas.core.series.Series

In [141]:
# now, for each age, calculate a ratio of transfer value to minutes played (a measure of current value)
# a high value here indicates the age group is artificially inflated in value
# a low value indicates the age group is under-valued per production right now

value_per_minute = age_val / age_mins
value_per_minute

age
16.0    14863.913656
17.0    20437.632884
18.0     8650.914272
19.0     5599.693983
20.0     4030.728799
21.0     2976.157423
22.0     2537.929120
23.0     2045.317992
24.0     1789.828931
25.0     1367.083936
26.0     1172.498111
27.0      835.546054
28.0      633.923454
29.0      420.238420
30.0      348.266369
31.0      243.096023
32.0      145.250593
33.0      106.682260
34.0       57.366323
35.0       47.475431
36.0       29.976249
37.0       24.268057
38.0       42.108908
39.0       14.969681
40.0        7.673839
41.0       16.628897
43.0       11.449508
dtype: float64

In [142]:
#given a series that converts age to value per minute, make a column that converts age to value per minute
this_year['value_per_minute'] = this_year['age'].map(value_per_minute)
this_year.head()

,player_id,date,market_value_in_eur_x,current_club_id_x,player_club_domestic_competition_id,first_name,last_name,name,last_season,current_club_id_y,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur_y,highest_market_value_in_eur,age,value_per_minute
0,745075,2024-01-10,25000,89,L1,David,Preu,David Preu,2024,89,...,2025-06-30 00:00:00,MGSC,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/david-preu/pro...,L1,1. FC Union Berlin,25000.0,50000.0,19.0,5599.693983
1,943560,2024-01-17,75000,873,GB1,Franco,Umeh-Chibueze,Franco Umeh-Chibueze,2023,873,...,2025-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/franco-umeh-ch...,GB1,Crystal Palace Football Club,75000.0,75000.0,18.0,8650.914272
2,590410,2024-01-17,75000,371,SC1,Josh,Clarke,Josh Clarke,2024,371,...,2027-06-30 00:00:00,CSG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/josh-clarke/pr...,SC1,The Celtic Football Club,75000.0,75000.0,19.0,5599.693983
3,352815,2024-01-17,150000,2420,PO1,Dénis,Duarte,Dénis Duarte,2023,2420,...,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/denis-duarte/p...,PO1,Vitória Sport Clube,150000.0,1000000.0,29.0,420.238420
4,703626,2024-01-17,100000,1237,GB1,Cameron,Peupion,Cameron Peupion,2023,1237,...,2025-06-30 00:00:00,Cove Agency,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/cameron-peupio...,GB1,Brighton and Hove Albion Football Club,100000.0,150000.0,21.0,2976.157423


In [143]:
this_year['current_value'] = this_year['market_value_in_eur_x'] / this_year['value_per_minute']

In [144]:
# sort the players by current value
this_year = this_year.sort_values('current_value', ascending=False)
this_year.head()

,player_id,date,market_value_in_eur_x,current_club_id_x,player_club_domestic_competition_id,first_name,last_name,name,last_season,current_club_id_y,...,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur_y,highest_market_value_in_eur,age,value_per_minute,current_value
7795,88755,2024-10-01,45000000,281,GB1,Kevin,De Bruyne,Kevin De Bruyne,2024,281,...,Roc Nation Sports,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/kevin-de-bruyn...,GB1,Manchester City Football Club,45000000.0,150000000.0,33.0,106.682260,421813.337965
5665,38253,2024-06-07,15000000,131,ES1,Robert,Lewandowski,Robert Lewandowski,2024,131,...,Gol International,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/robert-lewando...,ES1,Futbol Club Barcelona,15000000.0,90000000.0,35.0,47.475431,315952.896147
1051,132098,2024-05-29,100000000,27,L1,Harry,Kane,Harry Kane,2023,27,...,CK66,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/harry-kane/pro...,L1,FC Bayern München,100000000.0,150000000.0,30.0,348.266369,287136.539326
5612,125781,2024-06-07,25000000,13,ES1,Antoine,Griezmann,Antoine Griezmann,2024,13,...,BY AND FOR,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/antoine-griezm...,ES1,Club Atlético de Madrid S.A.D.,25000000.0,150000000.0,33.0,106.682260,234340.743314
655,95424,2024-05-27,13000000,281,GB1,Kyle,Walker,Kyle Walker,2024,281,...,CAA Base Ltd,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/kyle-walker/pr...,GB1,Manchester City Football Club,13000000.0,50000000.0,34.0,57.366323,226613.794126


Okay, so clearly the model is overestimating the value of old players, as Kyle Walker is certainly not the 5th best player in the world.  Perhaps using minutes wasn't the best tool to value player productivity.  Adjusting for position might also help here, as goalies are known to age slower than field players, perhaps skewing our data a bit here.  Lastly, fitting the function into a line of logarithmic best fit (which is what I expected value-per-minute to look like) might help even out the data a bit.  

Before we change the measure of performance (as this is a lot more complicated than the other two ideas), let's see if the other two methods come out with a result that looks more like generally accepted results.

In [145]:
appearances.head()
appearances.columns
appearances['position'].unique()

array(['Attack', 'Goalkeeper', 'Defender', 'Midfield', 'Missing'],
      dtype=object)

We're given Attack, Midfeild, Defender and Goalkeeper here, and while a more complex process may give more accurate results, this is a process that could be done later after we add more detailed statistical data.

In [146]:
age_mins = dict()
value_per_minute = dict()

for position in appearances['position'].unique():
    age_mins[position] = appearances[appearances['position'] == position].groupby('age')['minutes_played'].sum()
    print(position)
    print(age_mins[position])
    # in order for minutes to be an accurate representaion of value, we should scale the minutes by the total value of the players in that position
    pos_val = this_year[this_year['position'] == position]['market_value_in_eur_x'].sum()
    pos_mins = age_mins[position].sum()
    print(pos_val)
    age_mins[position] = age_mins[position] * (pos_val / pos_mins)



    value_per_minute[position] = age_val / age_mins[position]
    print(value_per_minute[position])





Attack
age
16.0        342.0
17.0       7534.0
18.0      18398.0
19.0      93031.0
20.0     121279.0
21.0     216752.0
22.0     419696.0
23.0     499795.0
24.0     764052.0
25.0     891444.0
26.0     993879.0
27.0    1100383.0
28.0    1118548.0
29.0     987844.0
30.0    1091778.0
31.0    1193018.0
32.0     887195.0
33.0     712060.0
34.0     529446.0
35.0     443918.0
36.0     225782.0
37.0     224534.0
38.0      76601.0
40.0       4968.0
Name: minutes_played, dtype: float64
13459325000
age
16.0    86.857024
17.0    43.075236
18.0    27.853663
19.0    14.203527
20.0    14.694177
21.0    12.104699
22.0     7.823027
23.0     7.077311
24.0     6.000802
25.0     4.524095
26.0     4.184279
27.0     2.985158
28.0     2.140184
29.0     1.729809
30.0     1.433413
31.0     0.960984
32.0     0.660788
33.0     0.473969
34.0     0.298553
35.0     0.206979
36.0     0.211003
37.0     0.117574
38.0     0.329025
39.0          NaN
40.0     0.344505
41.0          NaN
43.0          NaN
dtype: float64
Goa

C:\Users\Aaron\AppData\Local\Temp\ipykernel_21352\1300377572.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  age_mins[position] = age_mins[position] * (pos_val / pos_mins)


In [147]:


# for each set of value_per_minute data, perform logaritmic regression to smooth out the data
# this will allow us to predict the value per minute for each age group

from scipy.optimize import curve_fit
import statsmodels.api as sm
import matplotlib.pyplot as plt

new_values_per_minute = dict()

for position in value_per_minute:
    if position == 'Missing':
        continue
    print(position)
    print(value_per_minute[position])
    #first, iterate through the data, replacing any inf or nan values with the nearest real value
    for i in range(16, 16+len(value_per_minute[position])-1):
        if np.isnan(value_per_minute[position][i]):
            value_per_minute[position][i] = value_per_minute[position][i-1]
        if np.isinf(value_per_minute[position][i]):
            value_per_minute[position][i] = value_per_minute[position][i-1]

    # remove the 43.0 age
    for i in [43.0]:
        value_per_minute[position] = value_per_minute[position].drop(i) if i in value_per_minute[position] else value_per_minute[position]

    print(value_per_minute[position])

    x_data = value_per_minute[position].index
    y_data = value_per_minute[position].values
    print(x_data, y_data)
    
    # exponential fitting hasn't been working, let's get the logs and do a linear fit
    Y = np.log(y_data)

    # Add a constant to the independent variable for the intercept
    X = sm.add_constant(x_data)

    # Fit the regression model
    model = sm.OLS(Y, X).fit()


    # Calculate the residual standard deviation
    print(model.summary())
    
    # Now, using the model, calculate the predicted values delinearlized
    a, b = model.params
    c = np.exp(a)
    x_fit = np.linspace(min(x_data), max(x_data), 100)
    y_fit = c * np.exp(b * x_fit)


    # Plot the data and the fitted model
    # graph(x_data, y_data, x_fit, y_fit, a, b, c)

    new_value_per_minute = dict()
    for i in range(16, 44):
        new_value_per_minute[i] = c * np.exp(b * i)

    new_values_per_minute[position] = new_value_per_minute


Attack
age
16.0    86.857024
17.0    43.075236
18.0    27.853663
19.0    14.203527
20.0    14.694177
21.0    12.104699
22.0     7.823027
23.0     7.077311
24.0     6.000802
25.0     4.524095
26.0     4.184279
27.0     2.985158
28.0     2.140184
29.0     1.729809
30.0     1.433413
31.0     0.960984
32.0     0.660788
33.0     0.473969
34.0     0.298553
35.0     0.206979
36.0     0.211003
37.0     0.117574
38.0     0.329025
39.0          NaN
40.0     0.344505
41.0          NaN
43.0          NaN
dtype: float64
age
16.0    86.857024
17.0    43.075236
18.0    27.853663
19.0    14.203527
20.0    14.694177
21.0    12.104699
22.0     7.823027
23.0     7.077311
24.0     6.000802
25.0     4.524095
26.0     4.184279
27.0     2.985158
28.0     2.140184
29.0     1.729809
30.0     1.433413
31.0     0.960984
32.0     0.660788
33.0     0.473969
34.0     0.298553
35.0     0.206979
36.0     0.211003
37.0     0.117574
38.0     0.329025
39.0     0.329025
40.0     0.344505
41.0     0.344505
dtype: float64
I

In [ ]:
def graph(x_data,y_data, x_fit, y_fit, a, b, c):
    plt.scatter(x_data, y_data, label="Data", color="red")
    plt.plot(x_fit, y_fit, label=f"Best Fit: $y = {a:.2f} e^{{{b:.2f}x}} + {c:.2f}$", color="blue")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Exponential Curve Fit")
    plt.legend()
    plt.show()

In [148]:
#given a series that converts age to value per minute, make a column that converts age to value per minute
# but now, specifically account for position via the value_per_minute dictionary
# exclude players where position == 'Missing'

print(new_values_per_minute)
this_year = this_year[this_year['position'] != 'Missing']

this_year['value_per_minute'] = this_year.apply(lambda row: new_values_per_minute[row['position']][row['age']], axis=1)
this_year.head()

{'Attack': {16: 38.58271888281373, 17: 30.527182150173317, 18: 24.15352979297085, 19: 19.11060767384382, 20: 15.12057777037058, 21: 11.963610786837883, 22: 9.465774736426354, 23: 7.4894522195042095, 24: 5.925758441343707, 25: 4.6885422426102, 26: 3.709639631506079, 27: 2.935118312590759, 28: 2.322306305911458, 29: 1.8374409492596433, 30: 1.4538087561584996, 31: 1.1502736457107576, 32: 0.9101124576473966, 33: 0.7200935956879826, 34: 0.5697480373922573, 35: 0.4507925470468758, 36: 0.35667331370393507, 37: 0.28220487126934873, 38: 0.22328440707020913, 39: 0.17666571883201185, 40: 0.13978036630483384, 41: 0.1105961639501342, 42: 0.08750521839247748, 43: 0.06923534209891452}, 'Goalkeeper': {16: 1713.0074024736314, 17: 1144.103730612313, 18: 764.1375889624389, 19: 510.3612891401276, 20: 340.8661597271279, 21: 227.66173947651856, 22: 152.05342666742058, 23: 101.55524865296636, 24: 67.82792571668902, 25: 45.30172066979922, 26: 30.256651282785125, 27: 20.208171639236735, 28: 13.496873701723882,

KeyError: nan

In [129]:
# Filter out rows where 'position' or 'age' is NaN
this_year = this_year.dropna(subset=['position', 'age'])

# Apply the lambda function to calculate 'value_per_minute'
this_year['value_per_minute'] = this_year.apply(lambda row: new_values_per_minute[row['position']][row['age']], axis=1)

# Calculate 'current_value' by dividing 'market_value_in_eur_x' by 'value_per_minute'
this_year['current_value'] = this_year['market_value_in_eur_x'] / this_year['value_per_minute']
this_year.sort_values('current_value', ascending=False).head()  

# give me a list of the top 100 players, sorted by current value
top_100 = this_year.sort_values('current_value', ascending=False).head(100)
top_100['name'].to_list()


['Kevin De Bruyne',
 'Rodri',
 'Virgil van Dijk',
 'Bruno Fernandes',
 'Harry Kane',
 'Marquinhos',
 'Luka Modric',
 'Kyle Walker',
 'Bernardo Silva',
 'John Stones',
 'Hakan Çalhanoğlu',
 'Manuel Neuer',
 'Mohamed Salah',
 'Antonio Rüdiger',
 'Federico Valverde',
 'Rúben Dias',
 'Joshua Kimmich',
 'Andrew Robertson',
 'Nathan Aké',
 'Heung-min Son',
 'Kylian Mbappé',
 'Sergio Ramos',
 'Jesús Navas',
 'Nicolò Barella',
 'Raphaël Varane',
 'David Alaba',
 'Antoine Griezmann',
 'João Palhinha',
 'João Cancelo',
 'Manuel Akanji',
 'Benjamin Pavard',
 'Alejandro Grimaldo',
 'Toni Kroos',
 'Kieran Trippier',
 'Vinicius Junior',
 'Erling Haaland',
 'Harry Maguire',
 'Robert Lewandowski',
 'James Maddison',
 'Claudio Bravo',
 'Mateo Kovacic',
 'Rodrigo De Paul',
 'Declan Rice',
 'Gabriel Magalhães',
 'Mats Hummels',
 'İlkay Gündoğan',
 'Bruno Guimarães',
 'Lucas Hernández',
 'Leroy Sané',
 'Daniel Carvajal',
 'Lewis Dunk',
 'Jack Grealish',
 'Lautaro Martínez',
 'Martin Ødegaard',
 'Granit Xh